<a href="https://colab.research.google.com/github/AliKayhanAtay/Data-Visualization/blob/main/Altair_Interactive_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import altair as alt
import pandas as pd
import numpy as np
import os 
import re
from matplotlib.pyplot import *
%matplotlib inline
 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

In [ ]:
df = pd.read_csv(XX, encoding='ANSI', parse_dates=['DATA_DT'], sep=';').groupby(['DATA_DT','INTERNAL_ORG_RK']).head(100)
df = df.query('DATA_DT > "01-01-2018"')
df = df.merge(pd.read_excel('coordinates.xlsx'), on=['INTERNAL_ORG_RK'], how='left')

df['NAKDI_RISK_ORIG_CUR'] = df.apply(lambda row: row['NAKDI_RISK_USD'] if row['CURRENCY']!='TRY' else row['NAKDI_RISK_TL'], axis=1)
df['ECL_ORIG_CUR']        = df.apply(lambda row: row['ECL_YP_USD']     if row['CURRENCY']!='TRY' else row['ECL_YP_USD']   , axis=1)
df['CURRENCY_TP_YP']      = df.apply(lambda row: 'USD'                 if row['CURRENCY']!='TRY' else 'TRY'               , axis=1)

use_cols = {
            'DATA_DT'              : 'DATE',
            'INTERNAL_ORG_RK'      : 'BRANCH',
            'PRODUCT_TYPE_CD'      : 'PRODUCT_TYPE_DETAIL',
            'IFRS9_FINAL_STAGE'    : 'CLASS',
            'RETAIL_NONRETAIL_FLG' : 'COMMERCIAL/RETAIL',
            'CURRENCY'             : 'ORIGINAL_CURRENCY',
            'NAKDI_RISK_USD'       : 'RISK_USD',
            'NAKDI_RISK_TL'        : 'RISK_TL',
            'ECL_YP_USD'           : 'ECL_USD',
            'ECL_TL'               : 'ECL_TL',
            'NAKDI_RISK_ORIG_CUR'  : 'RISK',
            'ECL_ORIG_CUR'         : 'ECL',
            'CURRENCY_TP_YP'       : 'CURRENCY'
}
data = df[[e for e in use_cols]].rename(columns=use_cols)

In [ ]:
grouper = data['PRODUCT_TYPE_DETAIL'].str.split('_', expand=True)[0].rename({0:'ÜRÜN TİPİ DETAY'})
grouped = (data.groupby(grouper)['RISK_TL'].sum() / data['RISK_TL'].sum()).sort_values(ascending=False)
mapper = {e:e.split('_')[0] if e.split('_')[0] in grouped.head().index else 'OTHER' for e in data['PRODUCT_TYPE_DETAIL'].unique()}
data['PRODUCT_TYPE'] = data['PRODUCT_TYPE_DETAIL'].map(mapper)
data['DATE'] = data['DATE']+ pd.DateOffset(1)

In [ ]:
def MakePlot(data, column, TP=False, title='', save=False):
    # preparing data
    if TP:
        index = ['DATE', column]
        values  = ['RISK_TL']
    else:
        index = ['DATE', column, 'CURRENCY']
        values  = ['RISK']
    columns = None
    aggfunc = {e:'sum' for e in values}
    table = pd.pivot_table(data, index=index, columns=columns, values=values, aggfunc=aggfunc)
    table = (table / 1000000).reset_index()
    table = table.groupby(index, as_index=False)[values[0]].sum()

    if not TP:
        table[column]  = table[column] + ' ' + table['CURRENCY']
    
    ############################################################################################################
    ############################################################################################################
    ############################################################################################################
    
    nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=['DATE'])
    
    line = alt.Chart(table, title=title
                    ).mark_line(interpolate='basis'
                               ).encode(alt.X('DATE:T', axis=alt.Axis(format='%m-%Y', labelAngle=-55), title='DATE'),
                                        alt.Y(f'{values[0]}:Q'),
                                        color=f'{column}:N')
    
    selectors = alt.Chart(table).mark_point().encode(x='DATE:T',opacity=alt.value(0),).add_selection(nearest)

    points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

    text = line.mark_text(align='left', dx=20, dy=-20, fontSize=15
                     ).encode(text=alt.condition(nearest, f'{values[0]}:Q', alt.value(' '), format=',.0f'),)

    rules = alt.Chart(table).mark_rule(color='gray').encode(x='DATE:T',).transform_filter(nearest)

    chart = alt.layer(line, selectors, points, rules, text).properties(width=800, height=500)
    
    if save:
        chart.properties(width=1300, height=600).save('charts\\x.html')
    else:
        chart.display()

# Interactive Altair Charts 

In [ ]:
for col in ['COMMERCIAL/RETAIL','CLASS','PRODUCT_TYPE']:
    MakePlot(data, col, TP=True,  title=col + ' TP'    , save=False)
    MakePlot(data, col, TP=False, title=col + ' TP/TP', save=False)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)